In [86]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [87]:
tickers =  ["AAPL", "META", "CSCO", "3988.HK"]
income_statement_dict = {}
balance_sheet_dict = {}
cashflow_st_dict = {}

In [88]:
for ticker in tickers:
    #scraping income statement
    url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(ticker, ticker)
    income_statement = {}
    table_title = {}

    headers = {"User-Agent": "Chrome/117.0.5938.88"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,"html.parser")
    tabl = soup.find_all("div" , {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        heading = t.find_all("div" , {"class": "D(tbr) C($primaryColor)"})
        for top_row in heading:
            table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
        rows = t.find_all("div" , {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            income_statement[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[1:]
    temp = pd.DataFrame(income_statement).T
    temp.columns = table_title["Breakdown"]
    income_statement_dict[ticker] = temp
    

In [89]:
for ticker in tickers:
    #scraping balance sheet
    url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(ticker, ticker)
    balance_sheet = {}
    table_title = {}

    headers = {"User-Agent": "Chrome/117.0.5938.88"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,"html.parser")
    tabl = soup.find_all("div" , {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        heading = t.find_all("div" , {"class": "D(tbr) C($primaryColor)"})
        for top_row in heading:
            table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
        rows = t.find_all("div" , {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            balance_sheet[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[1:]
    temp = pd.DataFrame(balance_sheet).T
    temp.columns = table_title["Breakdown"]
    balance_sheet_dict[ticker] = temp
    

In [90]:
for ticker in tickers:
    #scraping cash flow
    url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(ticker, ticker)
    cash_flow = {}
    table_title = {}

    headers = {"User-Agent": "Chrome/117.0.5938.88"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,"html.parser")
    tabl = soup.find_all("div" , {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        heading = t.find_all("div" , {"class": "D(tbr) C($primaryColor)"})
        for top_row in heading:
            table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
        rows = t.find_all("div" , {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            cash_flow[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[1:]
    temp = pd.DataFrame(cash_flow).T
    temp.columns = table_title["Breakdown"]
    cashflow_st_dict[ticker] = temp
    

In [91]:
for ticker in tickers:
    for col in income_statement_dict[ticker].columns:
        income_statement_dict[ticker][col] = income_statement_dict[ticker][col].str.replace(',|- ','')
        income_statement_dict[ticker][col] = pd.to_numeric(income_statement_dict[ticker][col], errors = 'coerce')
        cashflow_st_dict[ticker][col] = cashflow_st_dict[ticker][col].str.replace(',|- ','')
        cashflow_st_dict[ticker][col] = pd.to_numeric(cashflow_st_dict[ticker][col], errors = 'coerce') 
        if col!="ttm": #yahoo has ttm column for income statement and cashflow statement only
            balance_sheet_dict[ticker][col] = balance_sheet_dict[ticker][col].str.replace(',|- ','')
            balance_sheet_dict[ticker][col] = pd.to_numeric(balance_sheet_dict[ticker][col], errors = 'coerce')